---
**Trabalho:** trabalho consistirá na análise dos casos da COVID19 em 5 cidades brasileiras

**Disciplina**: Linguagens de Programação para Ciência de Dados

**Discentes**: Bruna Mattioli de Oliveira e Gabriel Andrade Varga

**Descrição:** Criação do Banco de Dados no Postgre SQL e imputação dos dados oriundos de um arquivo CSV diretamente no Banco de Dados

---

---
***Pacotes necessários***

---

In [1]:
import psycopg2 as ps
import pandas.io.sql as sqlio
from datetime import datetime
from matplotlib import pyplot as plt
import csv
import pandas as pd
import numpy as np

---
***Importação do banco de dados***

---

Importação do conjunto de dados completo dos casos de COVID-19 no Brasil

In [2]:
covid_casos = pd.read_csv("G:/My Drive/Especialização/Disciplinas/Módulo 2/Linguagens de Programação para Ciência de Dados/Trabalho LPCD/Bases/caso_full.csv")

Filtro das 5 cidades necessárias para a análise dos dados: Atibaia, Osasco, São Paulo, Bragança Paulista e Franco da Rocha

In [5]:
covid_casos_filtro_cidade = covid_casos[(covid_casos['city'] == 'Atibaia') | (covid_casos['city'] == 'Osasco') | (covid_casos['city'] == 'Guarulhos') | (covid_casos['city'] == 'Bragança Paulista') | (covid_casos['city'] == 'Franco da Rocha')]

In [6]:
covid_casos_filtro_cidade.head(5)

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
1981046,Guarulhos,3518800.0,2020-03-17,202012,1392121.0,1379182.0,False,False,1,0.07183,2020-03-17,0.0,0,1,city,SP,1,0
1981059,Guarulhos,3518800.0,2020-03-18,202012,1392121.0,1379182.0,False,False,1,0.07183,2020-03-18,0.0,0,2,city,SP,0,0
1981062,Osasco,3534401.0,2020-03-18,202012,699944.0,698418.0,False,False,1,0.14287,2020-03-18,0.0,0,1,city,SP,1,0
1981076,Guarulhos,3518800.0,2020-03-19,202012,1392121.0,1379182.0,False,False,1,0.07183,2020-03-19,0.0,0,3,city,SP,0,0
1981080,Osasco,3534401.0,2020-03-19,202012,699944.0,698418.0,False,True,1,0.14287,2020-03-18,0.0,0,2,city,SP,0,0


In [7]:
covid_casos_filtro = covid_casos_filtro_cidade[(covid_casos_filtro_cidade['is_repeated'] == False)]

---
***Criação da conexão no BD (Postgre SQL)***

---

Conexão no postgre com o banco de dados Grupo_Beta

In [8]:
conn = ps.connect("dbname=grupo_beta user=postgres password=vagan9ch")
cur = conn.cursor()

---
***Criação das tabelas no BD com base no Modelo Relacional***

---

Criação das tabelas modeladas no Postgre

In [9]:
cur.execute("CREATE TABLE semana_epidemologica (id_sem_epid SERIAL PRIMARY KEY, semana INTEGER);")
cur.execute("CREATE TABLE data_dado (id_data_dado SERIAL PRIMARY KEY, data_dado DATE, id_sem_epid INTEGER);")
cur.execute("CREATE TABLE casos (id_casos SERIAL PRIMARY KEY, novos_casos INTEGER, id_cidade INTEGER, id_data_dado INTEGER);")
cur.execute("CREATE TABLE obitos (id_obitos SERIAL PRIMARY KEY, novos_obitos INTEGER, id_cidade INTEGER, id_data_dado INTEGER);")
cur.execute("CREATE TABLE populacao (id_populacao SERIAL PRIMARY KEY, pop_estimada_2019 INTEGER, pop_estimada VARCHAR, id_cidade INTEGER);")
cur.execute("CREATE TABLE cidade (id_cidade SERIAL PRIMARY KEY, nome_cidade VARCHAR, cod_cidade_ibge INTEGER);")

Atribuição das chaves estrangeiras

In [10]:
cur.execute("ALTER TABLE data_dado ADD FOREIGN KEY (id_sem_epid) REFERENCES semana_epidemologica (id_sem_epid);")
cur.execute("ALTER TABLE populacao ADD FOREIGN KEY (id_cidade) REFERENCES cidade (id_cidade);")
cur.execute("ALTER TABLE casos ADD FOREIGN KEY (id_cidade) REFERENCES cidade (id_cidade);")
cur.execute("ALTER TABLE casos ADD FOREIGN KEY (id_data_dado) REFERENCES data_dado (id_data_dado);")
cur.execute("ALTER TABLE obitos ADD FOREIGN KEY (id_cidade) REFERENCES cidade (id_cidade);")
cur.execute("ALTER TABLE obitos ADD FOREIGN KEY (id_data_dado) REFERENCES data_dado (id_data_dado);")

Executar as alterações no BD

In [ ]:
conn.commit()

---
***Imputando os dados da tabela / entidade semana epidemológica no BD***

---

* Criação da tabela apenas com os dados necessários

In [11]:
semana_epidemologica = covid_casos_filtro[['epidemiological_week']]

* Retirada dos valores duplicados. Não é necessário ter os dados repetidos no BD.

In [12]:
semana_epidemologica2 = semana_epidemologica.drop_duplicates(subset = 'epidemiological_week')

* Criação de uma lista contendo os dados da coluna epidemiological_week

In [13]:
epid_week = semana_epidemologica2['epidemiological_week'].tolist()

print(epid_week)

[202012, 202013, 202014, 202015, 202016, 202017, 202018, 202019, 202020, 202021, 202022, 202023, 202024, 202025, 202026, 202027, 202028, 202029, 202030, 202031, 202032, 202033, 202034, 202035, 202036, 202037, 202038, 202039, 202040, 202041, 202042, 202043, 202044, 202045, 202046, 202047, 202048, 202049, 202050, 202051, 202052, 202053, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108, 202109, 202110, 202111, 202112, 202113, 202114, 202115, 202116, 202117, 202118, 202119, 202120, 202121, 202122, 202123, 202124, 202125]


* Criação de uma lista de dicionários contendo os dados da coluna epidemiological_week e com a chave semana_epid

Esta lista de dicionários é necessária para poder executar o comando executemany do pacote psycopg2

In [14]:
semana_dicionario = {}
semana_lista = []

for i in range(0, len(epid_week)):
    
    semana_dicionario[i] = dict({'semana_epid':epid_week[i]})
    
    semana_lista.append(semana_dicionario[i])

* Imputação dos dados na tabela do BD semana_epidemologica

In [15]:
cur.executemany("""INSERT INTO semana_epidemologica (semana) VALUES (%(semana_epid)s);""", semana_lista)
    
conn.commit()

---
***Imputando os dados da tabela / entidade cidade no BD***

---

* Criação da tabela apenas com os dados necessários

In [16]:
cidade = covid_casos_filtro[['city', 'city_ibge_code']]

* Retirada dos valores duplicados. Não é necessário ter os dados repetidos no BD.

In [17]:
cidade2 = cidade.drop_duplicates(subset = 'city')

* Criação de uma lista contendo os dados da coluna epidemiological_week

In [18]:
nome_cidade = cidade2['city'].tolist()

print(nome_cidade)

cod_cidade = cidade2['city_ibge_code'].tolist()

print(cod_cidade)

['Guarulhos', 'Osasco', 'Franco da Rocha', 'Atibaia', 'Bragança Paulista']
[3518800.0, 3534401.0, 3516408.0, 3504107.0, 3507605.0]


* Criação de uma lista de dicionários contendo os dados da coluna city / city_ibge_code e com a chave cod_cidade / nm_cidade

Esta lista de dicionários é necessária para poder executar o comando executemany do pacote psycopg2

In [19]:
cod_cidade_dicionario = {}
cod_cidade_lista = []

for i in range(0, len(cod_cidade)):
    
    cod_cidade_dicionario[i] = dict({'cod_cidade':cod_cidade[i],'nm_cidade':nome_cidade[i]})
    
    cod_cidade_lista.append(cod_cidade_dicionario[i])

* Imputação dos dados na tabela do BD cidade

In [20]:
cur.executemany("""INSERT INTO cidade (nome_cidade, cod_cidade_ibge) VALUES (%(nm_cidade)s, (%(cod_cidade)s));""", cod_cidade_lista)
    
conn.commit()

---
***Imputando os dados da tabela / entidade data dado no BD***

---

* Criação da tabela apenas com os dados necessários

In [21]:
data_dados = covid_casos_filtro[['last_available_date', 'epidemiological_week']]

* Retirada dos valores duplicados. Não é necessário ter os dados repetidos no BD.

In [22]:
data_dados2 = data_dados.drop_duplicates(subset = 'last_available_date')

* Recuperação do conteúdo da base semana_epidemologica que foi imputada anteriormente no BD. Ela será utilizada para a captura da chave estrangeira id_sem_epid

In [23]:
sql = "SELECT * FROM semana_epidemologica;"
semana_epidemologica = sqlio.read_sql_query(sql, conn)

In [24]:
print(semana_epidemologica)

    id_sem_epid  semana
0             1  202012
1             2  202013
2             3  202014
3             4  202015
4             5  202016
..          ...     ...
62           63  202121
63           64  202122
64           65  202123
65           66  202124
66           67  202125

[67 rows x 2 columns]


* Cruzamento entre a base data_dados2 e a base semana_epidemologica para a captura da chave estrangeira id_sem_epid

In [25]:
data_dados3 = pd.merge(data_dados2, semana_epidemologica, right_on='semana', left_on='epidemiological_week', how='left')

In [26]:
print(data_dados3)

    last_available_date  epidemiological_week  id_sem_epid  semana
0            2020-03-17                202012            1  202012
1            2020-03-18                202012            1  202012
2            2020-03-19                202012            1  202012
3            2020-03-20                202012            1  202012
4            2020-03-25                202013            2  202013
..                  ...                   ...          ...     ...
432          2021-06-19                202124           66  202124
433          2021-06-20                202125           67  202125
434          2021-06-22                202125           67  202125
435          2021-06-23                202125           67  202125
436          2021-06-24                202125           67  202125

[437 rows x 4 columns]


* Criação de uma lista contendo os dados da last_available_date

In [27]:
last_available_date = data_dados3['last_available_date'].tolist()

* Criação de uma lista contendo os dados da coluna id_sem_epid

In [28]:
id_sem_epid = data_dados3['id_sem_epid'].tolist()

* Criação de uma lista de dicionários contendo os dados da coluna last_available_date / id_sem_epid e com a chave data_dado / id_sem_epid

Esta lista de dicionários é necessária para poder executar o comando executemany do pacote psycopg2

In [29]:
data_dados_dicionario = {}
data_dados_lista = []

for i in range(0, len(last_available_date)):
    
    data_dados_dicionario[i] = dict({'data_dado':last_available_date[i],'id_sem_epid':id_sem_epid[i]})
    
    data_dados_lista.append(data_dados_dicionario[i])

* Imputação dos dados na tabela do BD data_dado

In [30]:
cur.executemany("""INSERT INTO data_dado (data_dado, id_sem_epid) VALUES (%(data_dado)s, (%(id_sem_epid)s));""", data_dados_lista)
    
conn.commit()

---
***Imputando os dados da tabela / entidade população no BD***

---

* Criação da tabela apenas com os dados necessários

In [31]:
populacao = covid_casos_filtro[['estimated_population_2019', 'estimated_population', 'city']]

* Retirada dos valores duplicados. Não é necessário ter os dados repetidos no BD.

In [32]:
populacao2 = populacao.drop_duplicates(subset = 'city')

* Recuperação do conteúdo da base cidade que foi imputada anteriormente no BD. Ela será utilizada para a captura da chave estrangeira id_cidade

In [33]:
sql = "SELECT * FROM cidade;"
cidade = sqlio.read_sql_query(sql, conn)

In [34]:
print(cidade)

   id_cidade        nome_cidade  cod_cidade_ibge
0          1          Guarulhos          3518800
1          2             Osasco          3534401
2          3    Franco da Rocha          3516408
3          4            Atibaia          3504107
4          5  Bragança Paulista          3507605


* Cruzamento entre a base populacao2 e a base cidade para a captura da chave estrangeira id_cidade

In [35]:
populacao3 = pd.merge(populacao2, cidade, right_on='nome_cidade', left_on='city', how='left')

In [36]:
print(populacao3)

   estimated_population_2019  estimated_population               city  \
0                  1379182.0             1392121.0          Guarulhos   
1                   698418.0              699944.0             Osasco   
2                   154489.0              156492.0    Franco da Rocha   
3                   142761.0              144088.0            Atibaia   
4                   168668.0              170533.0  Bragança Paulista   

   id_cidade        nome_cidade  cod_cidade_ibge  
0          1          Guarulhos          3518800  
1          2             Osasco          3534401  
2          3    Franco da Rocha          3516408  
3          4            Atibaia          3504107  
4          5  Bragança Paulista          3507605  


* Criação de uma lista contendo os dados da estimated_population_2019

In [37]:
estimated_population_2019 = populacao3['estimated_population_2019'].tolist()

* Criação de uma lista contendo os dados da estimated_population

In [38]:
estimated_population = populacao3['estimated_population'].tolist()

* Criação de uma lista contendo os dados da id_cidade

In [39]:
id_cidade = populacao3['id_cidade'].tolist()

* Criação de uma lista de dicionários contendo os dados da coluna estimated_population_2019 / estimated_population / id_cidade e com a chave estimated_population_2019 / estimated_population / id_cidade

Esta lista de dicionários é necessária para poder executar o comando executemany do pacote psycopg2

In [40]:
populacao_dicionario = {}
populacao_lista = []

for i in range(0, len(id_cidade)):
    
    populacao_dicionario[i] = dict({'estimated_population_2019':estimated_population_2019[i],'estimated_population':estimated_population[i], 'id_cidade':id_cidade[i]})
    
    populacao_lista.append(populacao_dicionario[i])

* Imputação dos dados na tabela do BD populacao

In [41]:
cur.executemany("""INSERT INTO populacao (pop_estimada_2019, pop_estimada, id_cidade) VALUES (%(estimated_population_2019)s, (%(estimated_population)s), (%(id_cidade)s));""", populacao_lista)
    
conn.commit()

---
***Imputando os dados da tabela / entidade casos no BD***

---

* Criação da tabela apenas com os dados necessários

In [42]:
casos = covid_casos_filtro[['new_confirmed', 'city', 'last_available_date']]

* Recuperação do conteúdo da base cidade que foi imputada anteriormente no BD. Ela será utilizada para a captura da chave estrangeira id_cidade

In [43]:
sql_cidade = "SELECT * FROM cidade;"
cidade = sqlio.read_sql_query(sql_cidade, conn)

* Recuperação do conteúdo da base data_dado que foi imputada anteriormente no BD. Ela será utilizada para a captura da chave estrangeira id_data_dado

In [44]:
sql_data_dado = "SELECT * FROM data_dado;"
data_dado = sqlio.read_sql_query(sql_data_dado, conn)

* Cruzamento entre a base casos e a base cidade para a captura da chave estrangeira id_cidade

In [45]:
casos2 = pd.merge(casos, cidade, right_on='nome_cidade', left_on='city', how='left')

In [46]:
casos2.head(5)

,new_confirmed,city,last_available_date,id_cidade,nome_cidade,cod_cidade_ibge
0,1,Guarulhos,2020-03-17,1,Guarulhos,3518800
1,0,Guarulhos,2020-03-18,1,Guarulhos,3518800
2,1,Osasco,2020-03-18,2,Osasco,3534401
3,0,Guarulhos,2020-03-19,1,Guarulhos,3518800
4,0,Guarulhos,2020-03-20,1,Guarulhos,3518800


* Tratamento do campo de data para padronizar as duas chaves para o segundo cruzamento

In [47]:
casos2.last_available_date = pd.to_datetime(casos2.last_available_date)

In [48]:
casos2.last_available_date[0]

Timestamp('2020-03-17 00:00:00')

In [49]:
data_dado.data_dado = pd.to_datetime(data_dado.data_dado)

In [50]:
data_dado.data_dado[0]

Timestamp('2020-03-17 00:00:00')

* Cruzamento entre a base casos2 e a base data_dado para a captura da chave estrangeira id_data_dado

In [51]:
casos3 = pd.merge(casos2, data_dado, right_on='data_dado', left_on='last_available_date', how='left')

In [52]:
casos3.head(5)

,new_confirmed,city,last_available_date,id_cidade,nome_cidade,cod_cidade_ibge,id_data_dado,data_dado,id_sem_epid
0,1,Guarulhos,2020-03-17,1,Guarulhos,3518800,1,2020-03-17,1
1,0,Guarulhos,2020-03-18,1,Guarulhos,3518800,2,2020-03-18,1
2,1,Osasco,2020-03-18,2,Osasco,3534401,2,2020-03-18,1
3,0,Guarulhos,2020-03-19,1,Guarulhos,3518800,3,2020-03-19,1
4,0,Guarulhos,2020-03-20,1,Guarulhos,3518800,4,2020-03-20,1


* Criação de uma lista contendo os dados da new_confirmed

In [53]:
new_confirmed = casos3['new_confirmed'].tolist()

* Criação de uma lista contendo os dados da id_cidade

In [56]:
id_cidade = casos3['id_cidade'].tolist()

* Criação de uma lista contendo os dados da id_data_dado

In [54]:
id_data_dado = casos3['id_data_dado'].tolist()

* Criação de uma lista de dicionários contendo os dados da coluna new_confirmed / id_cidade / id_data_dado e com a chave new_confirmed / id_cidade / id_data_dado

Esta lista de dicionários é necessária para poder executar o comando executemany do pacote psycopg2

In [57]:
casos_dicionario = {}
casos_lista = []

for i in range(0, len(new_confirmed)):
    
    casos_dicionario[i] = dict({'new_confirmed':new_confirmed[i],'id_cidade':id_cidade[i], 'id_data_dado':id_data_dado[i]})
    
    casos_lista.append(casos_dicionario[i])

* Imputação dos dados na tabela do BD populacao

In [58]:
cur.executemany("""INSERT INTO casos (novos_casos, id_cidade, id_data_dado) VALUES (%(new_confirmed)s, (%(id_cidade)s), (%(id_data_dado)s));""", casos_lista)
    
conn.commit()

---
***Imputando os dados da tabela / entidade óbitos no BD***

---

* Criação da tabela apenas com os dados necessários

In [59]:
obitos = covid_casos_filtro[['new_deaths', 'city', 'last_available_date']]

* Recuperação do conteúdo da base cidade que foi imputada anteriormente no BD. Ela será utilizada para a captura da chave estrangeira id_cidade

In [60]:
sql_cidade = "SELECT * FROM cidade;"
cidade = sqlio.read_sql_query(sql_cidade, conn)

* Recuperação do conteúdo da base data_dado que foi imputada anteriormente no BD. Ela será utilizada para a captura da chave estrangeira id_data_dado

In [61]:
sql_data_dado = "SELECT * FROM data_dado;"
data_dado = sqlio.read_sql_query(sql_data_dado, conn)

* Cruzamento entre a base obitos e a base cidade para a captura da chave estrangeira id_cidade

In [62]:
obitos2 = pd.merge(obitos, cidade, right_on='nome_cidade', left_on='city', how='left')

* Tratamento do campo de data para padronizar as duas chaves para o segundo cruzamento

In [63]:
obitos2.last_available_date = pd.to_datetime(obitos2.last_available_date)

In [64]:
obitos2.last_available_date[0]

Timestamp('2020-03-17 00:00:00')

In [65]:
data_dado.data_dado = pd.to_datetime(data_dado.data_dado)

In [66]:
data_dado.data_dado[0]

Timestamp('2020-03-17 00:00:00')

* Cruzamento entre a base obitos2 e a base data_dado para a captura da chave estrangeira id_data_dado

In [67]:
obitos3 = pd.merge(obitos2, data_dado, right_on='data_dado', left_on='last_available_date', how='left')

In [68]:
obitos3.head(5)

,new_deaths,city,last_available_date,id_cidade,nome_cidade,cod_cidade_ibge,id_data_dado,data_dado,id_sem_epid
0,0,Guarulhos,2020-03-17,1,Guarulhos,3518800,1,2020-03-17,1
1,0,Guarulhos,2020-03-18,1,Guarulhos,3518800,2,2020-03-18,1
2,0,Osasco,2020-03-18,2,Osasco,3534401,2,2020-03-18,1
3,0,Guarulhos,2020-03-19,1,Guarulhos,3518800,3,2020-03-19,1
4,0,Guarulhos,2020-03-20,1,Guarulhos,3518800,4,2020-03-20,1


* Criação de uma lista contendo os dados da new_deaths

In [69]:
new_deaths = obitos3['new_deaths'].tolist()

* Criação de uma lista contendo os dados da id_cidade

In [70]:
id_cidade = obitos3['id_cidade'].tolist()

* Criação de uma lista contendo os dados da id_data_dado

In [71]:
id_data_dado = obitos3['id_data_dado'].tolist()

* Criação de uma lista de dicionários contendo os dados da coluna new_deaths / id_cidade / id_data_dado e com a chave new_deaths / id_cidade / id_data_dado

Esta lista de dicionários é necessária para poder executar o comando executemany do pacote psycopg2

In [72]:
obitos_dicionario = {}
obitos_lista = []

for i in range(0, len(new_deaths)):
    
    obitos_dicionario[i] = dict({'new_deaths':new_deaths[i],'id_cidade':id_cidade[i], 'id_data_dado':id_data_dado[i]})
    
    obitos_lista.append(obitos_dicionario[i])

* Imputação dos dados na tabela do BD obitos

In [73]:
cur.executemany("""INSERT INTO obitos (novos_obitos, id_cidade, id_data_dado) VALUES (%(new_deaths)s, (%(id_cidade)s), (%(id_data_dado)s));""", obitos_lista)
    
conn.commit()